In [269]:
import duckdb
import pandas as pd

In [270]:
#download data
exit_velocity_data = pd.read_csv('data/mlb-batter-exit-velocity.csv')

# run sql file to create tables
con = duckdb.connect('baseball_data.db').execute(open('baseball_data.sql').read())

# Create a DataFrame from a query
batting = con.execute("SELECT * FROM batting").df()

# Close the connection to the database
con.close()


In [271]:
exit_velocity_data['name'] = exit_velocity_data['player'].str.split(', ').str[1] + ' ' + exit_velocity_data['player'].str.split(', ').str[0].str.strip()

In [272]:
batting['name'] = batting['name'].str.strip()

In [273]:
# join the two dataframes on name and year
df = batting.merge(exit_velocity_data, how='inner', left_on=['name', 'Year'], right_on=['name', 'year'])

columns_to_drop = ["id", "rank", "year", "player"]

df.drop(columns_to_drop, axis=1, inplace=True)


In [274]:
#total bases
df['TB'] = df['H'] + df['2B'] + 2 * df['3B'] + 3 * df['HR']

#Bat Avg
df['AVG'] = (df['H'] / df['AB']).round(3) #, where=df['AB'] != 0

#on base
numerator_obp = df['H'] + df['BB'] + df['HBP']
denominator_obp = df['AB'] + df['BB'] + df['HBP'] + df['SF']
obp_calculation = numerator_obp / denominator_obp
df['OBP'] = obp_calculation.round(3)# , where=denominator_obp != 0

#slugging
slugging_calculation = df['TB'] / df['AB']
df['SLG'] = slugging_calculation.round(3)# , where=df['AB'] != 0

#on base + slugging 
df['OPS'] = (obp_calculation + slugging_calculation).round(3)

print("DataFrame with new stats columns (AVG, OBP, SLG, OPS):")
print(df[['name', 'Year', 'AB', 'H', 'TB', 'AVG', 'OBP', 'SLG', 'OPS']].tail())

DataFrame with new stats columns (AVG, OBP, SLG, OPS):
                  name  Year   AB    H   TB    AVG    OBP    SLG    OPS
903      Andrew Vaughn  2021  417   98  165  0.235  0.309  0.396  0.705
904  Christian Vázquez  2021  458  118  161  0.258  0.308  0.352  0.659
905   Christian Walker  2021  401   98  153  0.244  0.315  0.382  0.696
906     Patrick Wisdom  2021  338   78  175  0.231  0.305  0.518  0.823
907        Mike Zunino  2021  333   72  186  0.216  0.301  0.559  0.860


In [275]:
# batting = con.execute("SELECT * FROM batting").df()

stats_to_pivot = ['G', 'PA', 'AB', 'R', 'H', 'HR', 'RBI']


# - index: The column(s) to remain as rows (Player/Name)
# - columns: The column whose values will become the new column names (Year)
# - values: The columns whose values will populate the new wide columns (Selected Stats)
wide_batting_df = batting.pivot_table(
    index=['name'],           # one row per player
    columns='Year',           # Use Year values  as new columns
    values=stats_to_pivot,    # stats to pivot
    aggfunc='sum'             # Use 'sum' to aggregate if a player has multiple entries in a year,
).reset_index()

# Flatten MultiIndex columns
wide_batting_df.columns = ['{}_{}'.format(stat, year) if year else stat for stat, year in wide_batting_df.columns]

# Clean player names by removing extra spaces and special characters

wide_batting_df['name'] = wide_batting_df['name'].str.strip()
wide_batting_df['name'] = wide_batting_df['name'].str.replace('*', '')
wide_batting_df['name'] = wide_batting_df['name'].str.replace('#', '')




In [ ]:
# Take the first name all of its data and the data from the 2022 season if there are duplicates

wide_batting_df = wide_batting_df.groupby(by=('name')).sum()
wide_batting_df


,AB_2015,AB_2016,AB_2017,AB_2018,AB_2019,AB_2020,AB_2021,AB_2022,G_2015,G_2016,...,R_2021,R_2022,RBI_2015,RBI_2016,RBI_2017,RBI_2018,RBI_2019,RBI_2020,RBI_2021,RBI_2022
name,,,,,,,,,,,,,,,,,,,,,
A.J. Achter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Burnett,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Cole,2.0,7.0,14.0,3.0,0.0,0.0,0.0,0.0,3.0,7.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
A.J. Ellis,181.0,171.0,143.0,151.0,0.0,0.0,0.0,0.0,63.0,64.0,...,0.0,0.0,21.0,22.0,14.0,15.0,0.0,0.0,0.0,0.0
A.J. Griffin,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ángel Sánchez,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Óliver Pérez,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Óscar Hernández,31.0,11.0,0.0,0.0,0.0,4.0,0.0,0.0,18.0,4.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
